In [1]:
import pandas as pd

#### 1. 외부 파일 읽어오기

##### 1-1. CSV 파일
- csv(comma-separated values)
- 쉼표로 구분하고 줄바꿈으로 행 구분하는 텍스트 파일
- pandas.read_csv("파일경로(이름)", header = 0 , index_col = False)
    - csv 파일 읽어와서 데이터프레임으로 변환
    - header : 열 이름이 되는 행 지정
    - inex_col : 행 인덱스가 되는 열 지정

In [2]:
file_path = "./read_csv_sample.csv"

df1 = pd.read_csv(file_path)
print(df1)
print("\n")
# 파일 첫 행의 데이터가 열 이름이 됨.
df2 = pd.read_csv(file_path, header = None)
print(df2)
print("\n")
# 행 인덱스 정수로 자동 지정
df3 = pd.read_csv(file_path, index_col = None)
print(df3)
print("\n")
# 'c0'열이 행 인덱스가 됨
df4 = pd.read_csv(file_path, index_col = 'c0')
print(df4)

   c0  c1  c2  c3
0   0   1   4   7
1   1   2   5   8
2   2   3   6   9


    0   1   2   3
0  c0  c1  c2  c3
1   0   1   4   7
2   1   2   5   8
3   2   3   6   9


   c0  c1  c2  c3
0   0   1   4   7
1   1   2   5   8
2   2   3   6   9


    c1  c2  c3
c0            
0    1   4   7
1    2   5   8
2    3   6   9


##### 1-2. Excel 파일
- read_excel() 함수 인자롤 전달
- header 옵션 X -> 파일의 첫 행이 열 이름 구성
- header = None : 정수형 인덱스를 열 이름으로 자동 할당

In [3]:
df1 = pd.read_excel("./남북한발전전력량.xlsx") # header = 0 (default)
df2 = pd.read_excel("./남북한발전전력량.xlsx", header = None)

print(df1)
print("\n")
print(df2)

  전력량 (억㎾h) 발전 전력별  1990  1991  1992  1993  1994  1995  1996  1997  ...  2007  \
0        남한     합계  1077  1186  1310  1444  1650  1847  2055  2244  ...  4031   
1       NaN     수력    64    51    49    60    41    55    52    54  ...    50   
2       NaN     화력   484   573   696   803  1022  1122  1264  1420  ...  2551   
3       NaN    원자력   529   563   565   581   587   670   739   771  ...  1429   
4       NaN    신재생     -     -     -     -     -     -     -     -  ...     -   
5        북한     합계   277   263   247   221   231   230   213   193  ...   236   
6       NaN     수력   156   150   142   133   138   142   125   107  ...   133   
7       NaN     화력   121   113   105    88    93    88    88    86  ...   103   
8       NaN    원자력     -     -     -     -     -     -     -     -  ...     -   

   2008  2009  2010  2011  2012  2013  2014  2015  2016  
0  4224  4336  4747  4969  5096  5171  5220  5281  5404  
1    56    56    65    78    77    84    78    58    66  
2  2658  2802  

##### 1-3. JSON 파일
- json : 데이터 공유를 목적으로 개발된 특수한 파일 형식
- 딕셔너리와 비슷한 구조 key : value
- pd.read_json("파일경로(이름)")

In [4]:
df = pd.read_json("./read_json_sample.json")
print(df)
print("\n")
print(df.index)

           name  year        developer opensource
pandas           2008    Wes Mckinneye       True
NumPy            2006  Travis Oliphant       True
matplotlib       2003   John D. Hunter       True


Index(['pandas', 'NumPy', 'matplotlib'], dtype='object')


#### 2. 웹에서 가져오기

##### 2-1. HTML 웹 페이지에서 표 속성 가져오기
- read_html() : HTML 웹 페이지에 있는 table 태그에서 표 형식의 데이터 모두 찾아서 데이터프레임으로 변환
- 각각 별도의 데이터프레임으로 변환되므로 여러 개의 데이터프레임을 원소로 갖는 리스트 반환
- 표 정보 파싱
    - pd.read_html("https://www.naver.com/")

In [5]:
url = "./sample.html"

tables = pd.read_html(url)

# 표 개수 확인
print(len(tables))
print("\n")

# 테이블 출력
for i in range(len(tables)):
    print("tables[%s]" % i)
    print(tables[i])
    print("\n")

# 두 번째 데이터 프레임 df에 저장
df = tables[1]

# "name" 열 인덱스로 지정
df.set_index(["name"],inplace = True)
print(df)

2


tables[0]
   Unnamed: 0  c0  c1  c2  c3
0           0   0   1   4   7
1           1   1   2   5   8
2           2   2   3   6   9


tables[1]
         name  year        developer  opensource
0       NumPy  2006  Travis Oliphant        True
1  matplotlib  2003   John D. Hunter        True
2      pandas  2008    Wes Mckinneye        True


            year        developer  opensource
name                                         
NumPy       2006  Travis Oliphant        True
matplotlib  2003   John D. Hunter        True
pandas      2008    Wes Mckinneye        True


##### 2-2. 웹 스크래핑 

In [6]:
from bs4 import BeautifulSoup
import requests
import re
# 위키피디아 미국 ETF 웹 페이지 스크래핑 후 딕셔너리 형태로 저장
url = "https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'lxml')   
rows = soup.select('div > ul > li')
    
etfs = {}
for row in rows:
    
    try:
        etf_name = re.findall('^(.*) \(NYSE', row.text)
        etf_market = re.findall('\((.*)\|', row.text)
        etf_ticker = re.findall('NYSE Arca\|(.*)\)', row.text)
        
        if (len(etf_ticker) > 0) & (len(etf_market) > 0) & (len(etf_name) > 0):
            # 리스트를 원소로 갖는 딕셔너리 정의
            etfs[etf_ticker[0]] = [etf_market[0], etf_name[0]]

    except AttributeError as err:
        pass    

# etfs 딕셔너리 출력
print(etfs)
print('\n')

# etfs 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame(etfs)
print(df)

{}


Empty DataFrame
Columns: []
Index: []


#### 추가 자료

**- 웹 스크래핑 vs. 웹 크롤링**
- 웹 스크래핑 : 웹페이지에서 원하는 부분만 가져오는 것
- 웹 크롤링 : 웹페이지가 주어졌을 때, 그 페이지 내부의 링크를 따라가면서 모든 내용을 가져오는 것

**Request**
- HTML 문서 정보 가져오는데 사용

In [8]:
import requests
# res = requests.get("http://naver.com")
# 스터디 노션 주소
res = requests.get("https://www.notion.so/670f78fdee4a4e72a8003c55ad21e220")

# if문 대신 사용 가능
res.raise_for_status() # HTML 문서를 가져오지 못한 경우 에러 발생 & 프로그램 종료
print("웹 스크래핑을 진행합니다")

print(res.text)

웹 스크래핑을 진행합니다
<!doctype html><html class="notion-html"><head lang="en"><meta charset="utf-8"><meta name="viewport" content="width=device-width,height=device-height,initial-scale=1,maximum-scale=1,user-scalable=no,viewport-fit=cover"><title>Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.</title><meta name="description" content="A new tool that blends your everyday work apps into one. It's the all-in-one workspace for you and your team"><script>window.__webStartTime=Date.now(),window.__webStartTimePerformance=performance.now()</script><meta name="twitter:card" content="summary_large_image"><meta name="twitter:site" content="@NotionHQ"><meta name="twitter:title" content="Notion – The all-in-one workspace for your notes, tasks, wikis, and databases."><meta name="twitter:description" content="A new tool that blends your everyday work apps into one. It's the all-in-one workspace for you and your team"><meta name="twitter:url" content="https://www.notion.so"><me

**BeautifulSoup**
- 스크래핑 패키지
- lxml : 구문 분석 phaser

In [13]:
# 네이버 웹툰 페이지 예제
url = "https://comic.naver.com/webtoon/weekday"
res = requests.get(url)
res.raise_for_status()
# 가져온 html 문서 -> lmxl 파서를 통해서 beautiful soup 객체로 만듦.
soup = BeautifulSoup(res.text, "lxml") 
print(soup.title)
print("\n")
print(soup.title.get_text())
print("\n")
# soup 객체에서 첫번째로 발견되는 a태그 정보 출력
print(soup.a) 
print("\n")
# a element의 속성 , 딕셔너리로 반환
print(soup.a.attrs) 
print("\n")
# a element의 href 속성 '값' 정보 출력
print(soup.a["href"]) 
print("\n")
# 텍스트에 해당하는 a 태그 찾기
webtoon = soup.find("a", text = "재혼 황후-105화") 
print(webtoon)

<title>네이버 만화 &gt; 요일별  웹툰 &gt; 전체웹툰</title>


네이버 만화 > 요일별  웹툰 > 전체웹툰


<a href="#menu" onclick="document.getElementById('menu').tabIndex=-1;document.getElementById('menu').focus();return false;"><span>메인 메뉴로 바로가기</span></a>


{'href': '#menu', 'onclick': "document.getElementById('menu').tabIndex=-1;document.getElementById('menu').focus();return false;"}


#menu


<a href="/webtoon/detail?titleId=735661&amp;no=108" onclick="nclk_v2(event,'rnk*p.cont','735661','3')" title="재혼 황후-105화">재혼 황후-105화</a>
